In [7]:
from bs4 import BeautifulSoup
import requests
import csv
import unicodedata

In [ ]:
# base URL and construct weather URL
website = "https://www.timeanddate.com"
weather_url = f"{website}/weather/india"

# Fetch the webpage content
result = requests.get(weather_url)
format = result.text

# BeautifulSoup object for parsing HTML
parse = BeautifulSoup(format,'html')

# Find the weather table using its class
result_parse = parse.find("table", class_ = "zebra fw tb-wt zebra va-m")

# Function to clean text by removing non-ASCII characters
def clean_text(text):
    cleaned = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return cleaned.strip()

# Create and write to CSV file
with open('weather_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['City', 'Time', 'Temperature'])
    
    # Find all table rows
    rows = result_parse.find_all('tr')
    
    # Process each row to extract weather data
    for row in rows:
        city_link = row.find('a')
        if city_link:
            # Extract city name
            city_name = clean_text(city_link.text)
            
            # Extract time
            time_cell = row.find('td', id=lambda x: x and x.startswith('p'))
            time = clean_text(time_cell.text) if time_cell else None
            
            # Extract temperature
            temp_cell = row.find('td', class_='rbi')
            if temp_cell:
                temperature = clean_text(temp_cell.text).replace('\xa0', ' ')
                temperature = temperature.split()[0] + '°C'
            else:
                temperature = None
            
            writer.writerow([city_name, time, temperature])

    
    





